In [1]:
import logging
import numpy as np
import os

import torch
import torch.nn as nn

torch.manual_seed(42)
np.random.seed(42)

from tqdm import tqdm
import torch.optim as optim

from IPython.display import clear_output

In [2]:
def clear_env():
  !rm -fv *.py
  !rm -fv *.py*
  !rm -fv *.pl
  !rm -rfv H2-H2O/
  !rm -rfv src/

In [3]:
clear_env()

removed 'adahessian.py'
removed 'dataset.py'
removed 'genpip.py'
removed 'model.py'
removed 'dataset.py.1'
removed 'model.py.1'
removed 'postmsa.pl'
removed 'H2-H2O/MOL_2_2_1_3.BAS'
removed 'H2-H2O/MOL_2_2_1_3.POLY'
removed 'H2-H2O/MOL_2_2_1_3.MONO'
removed 'H2-H2O/MOL_2_2_1_3.FOC'
removed 'H2-H2O/points.dat'
removed 'H2-H2O/basis_2_2_1_3.f90'
removed 'H2-H2O/basis_2_2_1_3.so'
removed 'H2-H2O/MOL_2_2_1_3.MAP'
removed directory 'H2-H2O/'
removed 'src/polynomial.o'
removed 'src/Makefile'
removed 'src/molecule_simple.hh'
removed 'src/msa.o'
removed 'src/poly_basis.cpp'
removed 'src/utility.hh'
removed 'src/msa'
removed 'src/polynomial.hh'
removed 'src/monomial.hh'
removed 'src/monomial.o'
removed 'src/monomial.cpp'
removed 'src/polynomial.cpp'
removed 'src/msa.cpp'
removed directory 'src/'


In [4]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

ch = logging.StreamHandler()
formatter = logging.Formatter('[%(levelname)s] %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

In [5]:
!mkdir -v src/

!wget https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/src/Makefile
!wget https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/src/molecule_simple.hh
!wget https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/src/monomial.cpp
!wget https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/src/monomial.hh
!wget https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/src/msa.cpp
!wget https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/src/poly_basis.cpp
!wget https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/src/polynomial.cpp
!wget https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/src/polynomial.hh
!wget https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/src/utility.hh

!mv -v Makefile src/
!mv -v molecule_simple.hh src/
!mv -v monomial.cpp src/
!mv -v monomial.hh src/
!mv -v msa.cpp src/
!mv -v poly_basis.cpp src/
!mv -v polynomial.cpp src/
!mv -v polynomial.hh src/
!mv -v utility.hh src/

clear_output()

In [6]:
from zipfile import ZipFile

!wget https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/H2-H2O/points.zip

!mkdir -v H2-H2O

with ZipFile('points.zip', 'r') as f:
  f.extractall()

!mv -v points.dat H2-H2O/
!rm -vf points.zip

--2022-01-31 17:36:02--  https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/H2-H2O/points.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2794987 (2.7M) [application/zip]
Saving to: ‘points.zip’

points.zip          100%[===================>]   2.67M  --.-KB/s    in 0.02s   

2022-01-31 17:36:02 (109 MB/s) - ‘points.zip’ saved [2794987/2794987]

mkdir: created directory 'H2-H2O'
renamed 'points.dat' -> 'H2-H2O/points.dat'
removed 'points.zip'


In [7]:
!wget https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/genpip.py
!wget https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/postmsa.pl

from genpip import build_lib
from genpip import run_msa
from genpip import compile_dlib

logging.info("building MSA library...\n")
build_lib()

order = "3"
symmetry = "2 2 1"
wdir = "H2-H2O"
config_fname = "points.dat"

run_msa(order, symmetry, wdir)

!perl postmsa.pl H2-H2O 3 2 2 1
!gfortran -fPIC -shared H2-H2O/basis_2_2_1_3.f90 -o H2-H2O/basis_2_2_1_3.so

--2022-01-31 17:36:07--  https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/genpip.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3159 (3.1K) [text/plain]
Saving to: ‘genpip.py’

genpip.py           100%[===================>]   3.08K  --.-KB/s    in 0s      

2022-01-31 17:36:07 (57.7 MB/s) - ‘genpip.py’ saved [3159/3159]

--2022-01-31 17:36:07--  https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/postmsa.pl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6936 (6.8K) [text/plain]
Saving to: ‘postmsa.p

[INFO] building MSA library...

[INFO] g++ -O2 -Wall -c msa.cpp 
g++ -O2 -Wall -c monomial.cpp 
g++ -O2 -Wall -c polynomial.cpp 
g++ -O2 -Wall -o msa msa.o monomial.o polynomial.o
[INFO] cmd: ./src/msa 3 2 2 1
[INFO] 
[INFO] PIP basis files successfully generated.
[INFO] renamed 'MOL_2_2_1_3.BAS' -> 'H2-H2O/MOL_2_2_1_3.BAS'
[INFO] renamed 'MOL_2_2_1_3.FOC' -> 'H2-H2O/MOL_2_2_1_3.FOC'
[INFO] renamed 'MOL_2_2_1_3.MAP' -> 'H2-H2O/MOL_2_2_1_3.MAP'
[INFO] renamed 'MOL_2_2_1_3.MONO' -> 'H2-H2O/MOL_2_2_1_3.MONO'
[INFO] renamed 'MOL_2_2_1_3.POLY' -> 'H2-H2O/MOL_2_2_1_3.POLY'


In [25]:
# original github repo
# https://github.com/amirgholami/adahessian
# 
# lessw2020 update
# https://github.com/lessw2020/Best-Deep-Learning-Optimizers/tree/master/adahessian

#!wget https://raw.githubusercontent.com/lessw2020/Best-Deep-Learning-Optimizers/master/adahessian/adahessian.py
#from adahessian import Adahessian, get_params_grad

import math
import torch
from torch.optim.optimizer import Optimizer
from copy import deepcopy
import numpy as np

# imported from utils to avoid needing two imports... @lessw2020
def get_params_grad(model):
    """
    get model parameters and corresponding gradients
    """
    params = []
    grads = []
    for param in model.parameters():
        if not param.requires_grad:
            continue
        params.append(param)
        grads.append(0. if param.grad is None else param.grad + 0.)
    return params, grads

class Adahessian(Optimizer):
    """Implements Adahessian algorithm.
    It has been proposed in `ADAHESSIAN: An Adaptive Second OrderOptimizer for Machine Learning`.
    Arguments:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        lr (float, optional): learning rate (default: 0.15)
        betas (Tuple[float, float], optional): coefficients used for computing
            running averages of gradient and its square (default: (0.9, 0.999))
        eps (float, optional): term added to the denominator to improve
            numerical stability (default: 1e-4)
        weight_decay (float, optional): weight decay (L2 penalty) (default: 0)
        hessian_power (float, optional): Hessian power (default: 1)
    """

    def __init__(self, params, lr=0.15, betas=(0.9, 0.999), eps=1e-4,
                 weight_decay=0, hessian_power=1):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError(
                "Invalid beta parameter at index 0: {}".format(
                    betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError(
                "Invalid beta parameter at index 1: {}".format(
                    betas[1]))
        if not 0.0 <= hessian_power <= 1.0:
            raise ValueError("Invalid Hessian power value: {}".format(hessian_power))
        defaults = dict(lr=lr, betas=betas, eps=eps,
                        weight_decay=weight_decay, hessian_power=hessian_power)

        super(Adahessian, self).__init__(params, defaults)

    def get_trace(self, gradsH):
        """
        compute the Hessian vector product with a random vector v, at the current gradient point,
        i.e., compute the gradient of <gradsH,v>.
        :param gradsH: a list of torch variables
        :return: a list of torch tensors
        """

        params = self.param_groups[0]['params']

        v = [torch.randint_like(p, high=2) for p in params]
        for v_i in v:
            v_i[v_i == 0] = -1
        hvs = torch.autograd.grad(
            gradsH,
            params,
            grad_outputs=v,
            only_inputs=True,
            retain_graph=True)

        hutchinson_trace = []
        for hv, vi in zip(hvs, v):
            param_size = hv.size()
            if len(param_size) <= 2:  # for 0/1/2D tensor
                tmp_output = torch.abs(hv * vi)
                hutchinson_trace.append(tmp_output) # Hessian diagonal block size is 1 here.
            elif len(param_size) == 4:  # Conv kernel
                tmp_output = torch.abs(torch.sum(torch.abs(
                    hv * vi), dim=[2, 3], keepdim=True)) / vi[0, 1].numel() # Hessian diagonal block size is 9 here: torch.sum() reduces the dim 2/3.
                hutchinson_trace.append(tmp_output)
        
        return hutchinson_trace

    def step(self, gradsH, closure=None):
        """Performs a single optimization step.
        Arguments:
            gradsH: The gradient used to compute Hessian vector product.
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        # get the Hessian diagonal
        hut_trace = self.get_trace(gradsH)

        for group in self.param_groups:
            for i, p in enumerate(group['params']):
                if p.grad is None:
                    continue

                grad = deepcopy(gradsH[i].data)
                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state['step'] = 0
                    # Exponential moving average of gradient values
                    state['exp_avg'] = torch.zeros_like(p.data)
                    # Exponential moving average of Hessian diagonal square values
                    state['exp_hessian_diag_sq'] = torch.zeros_like(p.data)

                exp_avg, exp_hessian_diag_sq = state['exp_avg'], state['exp_hessian_diag_sq']

                beta1, beta2 = group['betas']

                state['step'] += 1

                # Decay the first and second moment running average coefficient
                exp_avg.mul_(beta1).add_(1 - beta1, grad)
                exp_hessian_diag_sq.mul_(beta2).addcmul_(
                    1 - beta2, hut_trace[i], hut_trace[i])

                bias_correction1 = 1 - beta1 ** state['step']
                bias_correction2 = 1 - beta2 ** state['step']

                # make the square root, and the Hessian power
                k = group['hessian_power']
                denom = (
                    (exp_hessian_diag_sq.sqrt() ** k) /
                    math.sqrt(bias_correction2) ** k).add_(
                    group['eps'])

                # make update
                p.data = p.data - \
                    group['lr'] * (exp_avg / bias_correction1 / denom + group['weight_decay'] * p.data)

        return loss


In [11]:
!wget https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/model.py
!wget https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/dataset.py

from dataset import PolyDataset
from dataset import HTOCM

clear_output()

In [12]:
wdir = './H2-H2O'
order = '3'
symmetry = '2 2 1'
dataset = PolyDataset(wdir=wdir, config_fname='points.dat', order=order, symmetry=symmetry)

[INFO] working directory: ./H2-H2O
[INFO] configuration file: ./H2-H2O/points.dat
[INFO] loading configurations...
[INFO] detected NATOMS = 5
[INFO] detected NCONFIGS = 44623
[INFO] preparing the coordinates..
[INFO] Done.
[INFO] detected NMON  = 39
[INFO] detected NPOLY = 102
[INFO] Preparing the polynomials...
[INFO] Done.


In [13]:
ids = np.random.permutation(len(dataset))
train_size = int(0.8 * len(dataset))
test_size  = len(dataset) - train_size
logging.info("Train size = {}".format(train_size))
logging.info("Test size  = {}".format(test_size))

train_ids = ids[:train_size]
test_ids  = ids[train_size:]

X, y             = dataset.X, dataset.y
X_train, y_train = X[train_ids], y[train_ids]
X_test, y_test   = X[test_ids], y[test_ids]

# scaling
mean = X_train.mean(axis=0)
std  = X_train.std(axis=0)
# first polynomial is a constant => std = 0.0
std[0] = 1.0

X_train = (X_train - mean) / std
X_test  = (X_test - mean) / std

[INFO] Train size = 35698
[INFO] Test size  = 8925


In [31]:
# Jun Li, Bin Jiang, and Hua Guo
# J. Chem. Phys. 139, 204103 (2013); https://doi.org/10.1063/1.4832697
# Suggest using Tanh activation function and 2 hidden layers

import torch.nn as nn

class FCNet(nn.Module):
    def __init__(self, NPOLY, n_layers=2, n_neurons=20, activation=nn.Tanh, init_form="uniform"):
        super().__init__()

        self.NPOLY      = NPOLY
        self.n_layers   = n_layers
        self.activation = activation()
        self.init_form  = init_form

        layers = [
            nn.Linear(self.NPOLY, n_neurons), self.activation,
            nn.Linear(n_neurons, n_neurons),         self.activation,
            nn.Linear(n_neurons, 1)
        ]

        self.layers = nn.Sequential(*layers)

        # if isinstance(self.activation, nn.ReLU):
        #     self.init_kaiming(activation_str="relu")
        # elif isinstance(self.activation, nn.Tanh):
        #     self.init_xavier(activation_str="tanh")
        # elif isinstance(self.activation, nn.Sigmoid):
        #     self.init_xavier(activation_str="sigmoid")
        # else:
        #     raise NotImplementedError()

    def forward(self, x):
        return self.layers(x)

    def init_xavier(self, activation_str):
        sigmoid_gain = nn.init.calculate_gain(activation_str)
        for child in self.layers.children():
            if isinstance(child, nn.Linear):
                for _ in range(0, self.n_layers - 1):
                    if self.init_form == "normal":
                        nn.init.xavir_normal_(child.weight, gain=sigmoid_gain)
                        if child.bias is not None:
                            nn.init.zeros_(child.bias)
                    elif self.init_form == "uniform":
                        nn.init.xavier_uniform_(child.weight, gain=sigmoid_gain)
                        if child.bias is not None:
                            nn.init.zeros_(child.bias)

                    else:
                        raise NotImplementedError()

    def init_kaiming(self, activation_str):
        raise NotImplementedError()

In [14]:
def count_params(model):
    nparams = 0
    for name, param in model.named_parameters():
        params = torch.tensor(param.size())
        nparams += torch.prod(params, 0)

    return nparams

In [23]:
def train(model, X_train, y_train, X_test, y_test, opt_type='adam', epochs=100, device='cpu'):
    # in case the structure of the model changes
    model.train()
    model.to(device)

    metric = nn.MSELoss().to(device)

    if opt_type == 'lbfgs':
        # Standard version from torch.optim
        optimizer = optim.LBFGS(model.parameters(), lr=1.5, max_iter=20, max_eval=None, tolerance_grad=1e-14, tolerance_change=1e-14, history_size=10)
    elif opt_type == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0)
    elif opt_type == 'adahessian':
        logging.info('Adahessian optimizer is chosen')
        optimizer = Adahessian(model.parameters(), lr=1.0, betas=(0.9, 0.999),
                               eps=1e-4, weight_decay=0.0, hessian_power=1.0)
    elif opt_type == 'sgd':
        optimizer = optim.SGD(model.parameters(), lr=1e-4)
    else:
        raise NotImplementedError()

    dprint = epochs // 20
  
    X_train.to(device)
    y_train.to(device)

    for epoch in tqdm(range(epochs)):
        optimizer.zero_grad()
        y_pred = model(X_train)

        loss = torch.sqrt(metric(y_pred, y_train))
        loss.backward(create_graph=True)
        
        _, gradsH = get_params_grad(model)

        optimizer.step(gradsH)

        with torch.no_grad():
            y_pred = model(X_test)
            test_loss = metric(y_pred, y_test)
            test_rmse = np.sqrt(test_loss.item())

        if epoch % dprint == 0:
            print("Epoch: {}; train rmse: {:.10f}; test rmse: {:.10f}".format(
                epoch, loss * HTOCM, test_rmse * HTOCM
            ))

In [26]:
n_layers   = 2
activation = nn.Tanh
init_form  = "uniform"
logging.info("Creating a fully connected neural network:")
logging.info("    n_layers   = {}".format(n_layers))
logging.info("    activation = {}".format(activation))
logging.info("    init_form  = {}".format(init_form))

model = FCNet(NPOLY=dataset.NPOLY, n_layers=n_layers, activation=activation, init_form=init_form)
model.double() # use double precision for model parameters 

nparams = count_params(model)
logging.info("number of parameters: {}".format(nparams))

train(model, X_train, y_train, X_test, y_test, opt_type='adahessian', epochs=10000)

[INFO] Creating a fully connected neural network:
[INFO]     n_layers   = 2
[INFO]     activation = <class 'torch.nn.modules.activation.Tanh'>
[INFO]     init_form  = uniform
[INFO] number of parameters: 2501
[INFO] Adahessian optimizer is chosen
  0%|          | 3/10000 [00:00<16:28, 10.11it/s]

Epoch: 0; train rmse: 283285.4689068816; test rmse: 482552.5886767792


  5%|▌         | 503/10000 [00:45<14:33, 10.87it/s]

Epoch: 500; train rmse: 920.6484789673; test rmse: 2548.4132577104


 10%|█         | 1002/10000 [01:28<12:18, 12.18it/s]

...

 90%|█████████ | 9003/10000 [12:51<01:24, 11.74it/s]

Epoch: 9000; train rmse: 542.3913803167; test rmse: 1223.2910132473


 95%|█████████▌| 9503/10000 [13:34<00:41, 11.88it/s]

Epoch: 9500; train rmse: 539.0658222682; test rmse: 1055.0151993697


100%|██████████| 10000/10000 [14:17<00:00, 11.66it/s]


In [30]:
n_layers   = 3
activation = nn.Tanh
init_form  = "uniform"
logging.info("Creating a fully connected neural network:")
logging.info("    n_layers   = {}".format(n_layers))
logging.info("    activation = {}".format(activation))
logging.info("    init_form  = {}".format(init_form))

model = FCNet(NPOLY=dataset.NPOLY, n_layers=n_layers, activation=activation, init_form=init_form)
model.double() # use double precision for model parameters 

nparams = count_params(model)
logging.info("number of parameters: {}".format(nparams))

train(model, X_train, y_train, X_test, y_test, opt_type='adahessian', epochs=10000)

[INFO] Creating a fully connected neural network:
[INFO]     n_layers   = 3
[INFO]     activation = <class 'torch.nn.modules.activation.Tanh'>
[INFO]     init_form  = uniform
[INFO] number of parameters: 2501
[INFO] Adahessian optimizer is chosen
  0%|          | 2/10000 [00:00<14:47, 11.27it/s]

Epoch: 0; train rmse: 40359.0259279137; test rmse: 1028075.2092588076


  5%|▌         | 502/10000 [00:42<13:40, 11.58it/s]

Epoch: 500; train rmse: 1424.7898992950; test rmse: 1961.0768309460


 10%|█         | 1002/10000 [01:25<12:54, 11.62it/s]

...

 90%|█████████ | 9002/10000 [12:58<01:24, 11.79it/s]

Epoch: 9000; train rmse: 401.1724953494; test rmse: 1135.1621155927


 95%|█████████▌| 9502/10000 [13:42<00:42, 11.63it/s]

Epoch: 9500; train rmse: 397.2644096662; test rmse: 1133.2381568767


100%|██████████| 10000/10000 [14:25<00:00, 11.56it/s]


In [32]:
n_layers   = 3
activation = nn.Tanh
init_form  = "uniform"
logging.info("Creating a fully connected neural network:")
logging.info("    n_layers   = {}".format(n_layers))
logging.info("    activation = {}".format(activation))
logging.info("    init_form  = {}".format(init_form))

model = FCNet(NPOLY=dataset.NPOLY, n_layers=n_layers, n_neurons=25, activation=activation, init_form=init_form)
model.double() # use double precision for model parameters 

nparams = count_params(model)
logging.info("number of parameters: {}".format(nparams))

train(model, X_train, y_train, X_test, y_test, opt_type='adahessian', epochs=10000)

[INFO] Creating a fully connected neural network:
[INFO]     n_layers   = 3
[INFO]     activation = <class 'torch.nn.modules.activation.Tanh'>
[INFO]     init_form  = uniform
[INFO] number of parameters: 3251
[INFO] Adahessian optimizer is chosen
  0%|          | 2/10000 [00:00<21:02,  7.92it/s]

Epoch: 0; train rmse: 64640.6955303114; test rmse: 299222.7980895268


  5%|▌         | 502/10000 [00:59<23:06,  6.85it/s]

Epoch: 500; train rmse: 1031.6693259233; test rmse: 2366.4363387110


 10%|█         | 1002/10000 [01:58<18:12,  8.23it/s]

...

 90%|█████████ | 9002/10000 [17:21<01:48,  9.24it/s]

Epoch: 9000; train rmse: 247.3765115994; test rmse: 1693.4420061411


 95%|█████████▌| 9502/10000 [18:17<00:55,  9.02it/s]

Epoch: 9500; train rmse: 243.6326660844; test rmse: 1738.0347462144


100%|██████████| 10000/10000 [19:12<00:00,  8.67it/s]
